## for all outdoor activity and ice skating

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait         # driver to control chrome browser
import pyautogui                            
from bs4 import BeautifulSoup as bs                               # to parse the html code
import threading                                                  # to do multi threding
from lxml import html
import time 
import pandas as pd   
import csv  

def get_driver():
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("--disable-logging")
    options.add_argument('log-level=3')
    options.add_argument('--disable-gpu')
    options.add_argument("--start-maximized")
    options.add_argument("--disable-notifications")
    options.add_argument("--disable-infobars")

    return webdriver.Chrome(executable_path=r'chromedriver.exe', options=options)

def write_output(filename, data): # create new csv
    with open(filename, mode="w", newline='',encoding='UTF-8') as out_file:
        writer = csv.writer(out_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
        writer.writerow(['Name','Rating','No Review','Address','Timing'])
        for row in data:
            writer.writerow(row)

driver = get_driver()

def process(url):
    driver.get(url)
    time.sleep(3)
    
    while(True):
        next_btn = driver.find_element_by_xpath('//*[@id="n7lv7yjyC35__section-pagination-button-next"]')
        try:
            for i in range(1,21):
                p = "ZXf8if5__or:{0}".format(i).replace('""',"''")
                st = "//div[@data-section-id='{0}']".format(p)
                each_review = driver.find_element_by_xpath(str(st))
                each_review.click()
                time.sleep(3)

                soup = bs(driver.page_source, "lxml")

                try:
                    name = soup.find("h1",{"class":"section-hero-header-title-title GLOBAL__gm2-headline-5"}).find("span").text
                except:
                    name = "N/A"
                try:
                    no_review = soup.find("button",{"class":"widget-pane-link"}).text.replace("(","").replace(")","").replace(",","")
                except:
                    no_review = "N/A"
                try:
                    rating = soup.find("span",{"class":"section-star-display"}).text
                except:
                    rating = "N/A"
                try:
                    address = soup.find("div", {"class": "ugiz4pqJLAG__primary-text gm2-body-2"}).text
                except:
                    address = "N/A"
                
                timing = {}
                try:
                    btn = driver.find_element_by_xpath('//span[@class="cX2WmPgCkHi__section-open-hours-button-right section-open-hours-button cX2WmPgCkHi__expand-more"]').click() 
                    soup1 = bs(driver.page_source, "lxml")
                    all_time = soup1.find_all("tr", {"class": "lo7U087hsMA__row-row"})
                    
                    for i in all_time:
                        timing[i.text.split('   ')[0].replace(' ', '')] = i.text.split('   ')[1]
                except:
                    timing["time"] = "Not given"

                info = []
                info.append(name)
                info.append(rating)
                info.append(no_review)
                info.append(address)
                info.append(timing)
                yield(info)

                print(info)
                back = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/button').click()
                time.sleep(2)

            next_btn.click()
            time.sleep(3)
        except:
            next_btn = driver.find_element_by_xpath('//*[@id="n7lv7yjyC35__section-pagination-button-next"]').click()
            time.sleep(3)

if __name__ == "__main__":
    url = ["https://www.google.com/maps/search/outdoor+activities+in+british+columbia/@50.6131926,-126.920475,4z/data=!3m1!4b1", "https://www.google.com/maps/search/ice+skating+in+british+columbia/@52.0579407,-126.344865,6z/data=!3m1!4b1"]
    filename = ["data1.csv", "data2.csv"]
    for i, filename in zip(url,filename):
        data = process(i)
        write_output(filename, data)

## scraping all reviews of each and every listed shop

In [ ]:
import selenium
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

import random

from bs4 import BeautifulSoup 
import re as r
import os
import time
import json
import csv

#this part of the code below chooses a header at random from a collection of 10 headers
header_list=[{'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.157 Safari/537.36COOKIE: 5=2; ax=v167-7'},
       {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/74.0.3729.157 Safari/537.36'},
       {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/79.0.3945.88 Safari/537.36'},
       {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.87 Safari/537.36'},
       {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36'},
       {'User-Agent':'Mozilla/5.0 (X11; Datanyze; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36'},
       {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.24 Safari/537.36'}]


def head():
    n = random.random()
    n=int(n*7)
    return header_list[n]

def scrape(address):
    path_chrome = r"chromedriver"
    ua = head()
    user_agent = "user-agent="+ua['User-Agent']
    chrome_options= webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument(user_agent)
    print('user-agent\n',user_agent)
    driver = webdriver.Chrome(path_chrome,options=chrome_options)
    print('opening defualt page')
    url  = 'https://www.google.com/maps/@30.7678096,76.4684188,14z'
    
    driver.get(url)

    element = driver.find_element_by_xpath("//input[@id='searchboxinput']")
    search  = driver.find_element_by_xpath("//button[@id='searchbox-searchbutton']")
    element.clear()
    element.send_keys(address)
    search.click()

    time.sleep(15)
    button_status = 0
    for status in range(1,6):
        print('trying to press SEE ALL REVIEWS button, attempt {}'.format(status))
        if(button_status !=1):
            try:
                wait = WebDriverWait(driver, 15)
                wait.until(EC.element_to_be_clickable(
                (By.XPATH, "//button[@class='allxGeDnJMl__button allxGeDnJMl__button-text']"))).click()
                print('button pressed: see all reviews ')
                button_status = 1
            except:
                continue
                print('button not found')
        if(button_status==1):
            break
        time.sleep(5)

    time.sleep(1)
    response = BeautifulSoup(driver.page_source, 'html.parser')
    com_ = response.find_all('div', class_ = 'jqnFjrOWMVU__root')[0].text
    com = com_.split(' ')
    combined_ratings = com[47]
    try:
        total_reviews = int(com[49].replace(',',''))
    except:
        total_reviews = int(com[50].replace(',',''))
    tags  = response.find_all('span', class_ = 'tuPVDR7ouq5__text gm2-body-2')
    count = response.find_all('span', class_ = 'tuPVDR7ouq5__subtext gm2-caption')
    tags_filtered = tags[1:-1]
    tags_list = []
    for i in range(len(tags_filtered)):
        tags_list.append(tags_filtered[i].text)
    count_list=[]
    for i in count:
        count_list.append(i.text)
    tags_dict = dict(zip(tags_list,count_list))

    print(combined_ratings)
    print(total_reviews)

    fname = "data.json"
    if os.path.isfile(fname):
        # File exists
        with open(fname, 'a+') as outfile:
            outfile.write(',')
            tags  = response.find_all('span', class_ = 'tuPVDR7ouq5__text gm2-body-2')
            count = response.find_all('span', class_ = 'tuPVDR7ouq5__subtext gm2-caption')
            tags_filtered = tags[1:-1]
            tags_list = []
            for i in range(len(tags_filtered)):
                tags_list.append(tags_filtered[i].text)
            count_list=[]
            for i in count:
                count_list.append(i.text)
            tags_dict = dict(zip(tags_list,count_list))
            
            ll = str(driver.current_url)
            i = ll.find('@')
            url = ll[i+1:].split(',')
            
            lattitude = url[0]
            longitude = url[1]
            
            data = {}
            data['address'] = address
            data['combined_ratings'] = combined_ratings
            data['total_reviews'] = total_reviews
            data['tags_dict'] = tags_dict
            data['lattitude'] = lattitude
            data['longitude'] = longitude
            
            json.dump(data, outfile)

    print('got tags and combined reviews. You can check the directory')
    print('Pressing scroll button')
    #int(total_reviews/12)  for final code
    #x = int(total_reviews/35) 
    x = 10
    for i in range(10):
        print('progress {}/{}'.format(i,x))
        try:
            elem = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "[class='section-loading-spinner']")))
            driver.execute_script("arguments[0].scrollIntoView();",elem)
        except Exception:
            break
        try:
            for see_more in wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "button[class^='section-expand-review']"))):
                see_more.click()
        except:
            continue
    time.sleep(2)
    path = r'C:\Users\RMohit\Downloads\The Travel Back\Web Scraping\Task 6 - google map scraping\temp_storage'
    filename = os.path.join(path,address+"_reviews.csv")
    if not os.path.exists(filename):
        with open(filename, "w", encoding='utf8',newline="\n") as f:
            csvwriter=csv.writer(f)
            csvwriter.writerow(["name","rating","review","time"])    
            response = BeautifulSoup(driver.page_source, 'html.parser')

            name_    = response.find_all('div', class_='section-review-title')
            ratings_ = response.find_all('span', class_='section-review-stars')
            reviews_ = response.find_all('div', class_ = 'section-review-review-content')
            time_    = response.find_all('div', class_ = 'section-review-metadata section-review-metadata-with-note')

            for i in range(len(name_)):
                nam = name_[i].text
                rat = ratings_[i]
                rat = rat['aria-label']
                rav = reviews_[i].text
                tim = time_[i].text
                csvwriter.writerow([nam,rat,rav,tim])
    print('execution complete, closing the chrome')
    time.sleep(5)
    chrome_options.arguments.remove(user_agent)
    driver.close()

filename = ["data1.csv","data2.csv"]
for i in filename:
    restaurant_data = pd.read_csv(i)
    restaurant_data['keyword'] = restaurant_data['Name'] +" " + restaurant_data['Address']
    keywords_list = list(restaurant_data['keyword'])
    l = len(keywords_list)
    print(l)
    for i in range(0,l):
        key = keywords_list[i]
        print(key)
        try:
            scrape(key)
        except:
            continue

## google map image scraping

In [ ]:
import selenium
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

import random

from bs4 import BeautifulSoup 
import re as r
import os
import time
import numpy as np
import csv
import cssutils

#this part of the code below chooses a header at random from a collection of 10 headers
header_list=[{'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.157 Safari/537.36COOKIE: 5=2; ax=v167-7'},
       {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/74.0.3729.157 Safari/537.36'},
       {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/79.0.3945.88 Safari/537.36'},
       {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.87 Safari/537.36'},
       {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36'},
       {'User-Agent':'Mozilla/5.0 (X11; Datanyze; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36'},
       {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.24 Safari/537.36'}]


def head():
    n = random.random()
    n=int(n*7)
    return header_list[n]

def scrape(address):
    path_chrome = r"chromedriver"
    ua = head()
    user_agent = "user-agent="+ua['User-Agent']
    chrome_options= webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument(user_agent)
    print('user-agent\n',user_agent)
    driver = webdriver.Chrome(path_chrome,options=chrome_options)
    print('opening defualt page')
    url  = 'https://www.google.com/maps/@30.7678096,76.4684188,14z'
    
    driver.get(url)

    element = driver.find_element_by_xpath("//input[@id='searchboxinput']")
    search  = driver.find_element_by_xpath("//button[@id='searchbox-searchbutton']")
    element.clear()
    element.send_keys(address)
    search.click()

    time.sleep(15)
    
    all_img = driver.find_element_by_xpath("//div[@class='section-carouselphoto-photo-container-shim']").click()
    time.sleep(5)
    
    soup = BeautifulSoup(driver.page_source, "lxml")
    
    #images
    results = soup.findAll('div', {'class': 'gallery-image-low-res'})
    print(len(results))
    j=0
    if(len(results) > 0):
        os.mkdir("images/{0}".format(address))
    for i in results:
        i = """{0}""".format(i)
        soup1 = BeautifulSoup(i,'html.parser')
        div_style = soup1.find('div')['style']
        style = cssutils.parseStyle(div_style)
        url = style['background-image'].replace('url(', '').replace(')', '')
        try:
            index = url.find('=')
            url = url[:index] + "=w1080-h1080-k-no"
        except:
            continue
        #print(url)
        try:
            urllib.request.urlretrieve(url, os.path.join("images", address , "image"+str(j)+'.jpg'))
        except:
            continue
        j += 1

    driver.close()
    
restaurant_data = pd.read_csv('data1.csv')
restaurant_data['keyword'] = restaurant_data['Name'] +" " + restaurant_data['Address']
keywords_list = list(restaurant_data['keyword'])
l = len(keywords_list)
print(l)
for i in range(0,l):
    key = keywords_list[i]
    print(key)
    try:
        scrape(key)
    except:
        continue